In [1]:
using SparseArrays, LinearAlgebra

function readgraph(graph)
    I=Int[]
    J=Int[]
    open(graph) do f
        linecounter = 0
        for l in eachline(f)
            v = split(l)
            if v[1] == "D"
                i,j = extrema((parse(Int, v[2])+1, parse(Int, v[3])+1))
                push!(I, i)
                push!(J, j)
            end
        end
    end
    N = maximum(I)
    H = sparse(J.-N,I,fill(1,length(I))) .÷ 2
end


function readseeds(seeds, H)
    M, N = size(H)
    @show size(H)
    v = Int[]
    open(seeds) do f
        for l in eachline(f)
            if l[1] != '#'
                v = parse.(Int, split(l)) .- N .+ 1
            end
        end
        println("$(length(v)) removed factors $v")
        return H[setdiff(1:size(H,1), v), :]
    end
end

readseeds (generic function with 1 method)

In [2]:
function leaf_removal(H::SparseMatrixCSC)
    M, N = size(H)
    Ht = sparse(transpose(H)) # for faster access by row
    degs = vec(sum(H .!= 0, dims=1))
    facts = fill(true, M)
    rowperm = Int[]
    Q = findall(degs .== 1)
    indep = findall(degs .== 0)
    dep = Int[]
    while !isempty(Q)
        i = popfirst!(Q)
        degs[i] == 0 && continue
        push!(dep, i)
        ∂i = rowvals(H)[nzrange(H,i)]
        ∂i = ∂i[facts[∂i]]
        @assert length(∂i) == 1 # should be a residual leaf
        a = ∂i[1]
        facts[a] = false
        push!(rowperm, a) 
        for j in rowvals(Ht)[nzrange(Ht,a)]
            degs[j] -= 1
            if j != i
                if degs[j] == 0
                    push!(indep, j)
                elseif degs[j] == 1
                    push!(Q, j)
                end
            end
        end
    end
    all(degs .==  0) || @warn "non-empty core"
    rowperm, [dep; indep]
end

leaf_removal (generic function with 1 method)

In [3]:
isuppertriang(H::SparseMatrixCSC) = all(rowvals(H)[last(nzrange(H,i))] == i for i = 1:size(H,1))

function ut2diagGF2!(T::SparseMatrixCSC)
    (m,n) = size(T)
    # Check that the left part of T is unit upper triangular
    @assert isuppertriang(T)
    # Loop over diagonal elements
    for c in m:-1:2
        # Find non-zero elements above T[c,c] and perform row operations to 
        #  cancel them out
        for (j,v) in @views zip(rowvals(T[:,c]),nonzeros(T[:,c]))
            if v != 0 && j < c
                for k in rowvals(T[c,c:end]).+c.-1
                    T[j,k] ⊻= T[c,k]
                end
            end
        end
    end
    dropzeros!(T)
end

ut2diagGF2! (generic function with 1 method)

In [4]:
function findbasis(H)
    rowperm, colperm = leaf_removal(H)
    Hnew = H[rowperm, colperm]
    ut2diagGF2!(Hnew)
    B = [Hnew[:, size(Hnew, 1)+1:end]; I]
    indep = colperm[size(H,1)+1:end]
    B[invperm(colperm),:], indep
end 

findbasis (generic function with 1 method)

In [5]:
H = readgraph("/home/ab/src/spread/graphN9000.txt")
H = readseeds("/home/ab/src/spread/seedsN9000.txt", H)
B, indep = findbasis(H);
M,N = size(H)
α=1-M/N

size(H) = (6600, 9000)
5 removed factors [129, 1515, 1531, 1655, 3471]


0.26722222222222225

In [6]:
rowperm,colperm = leaf_removal(H)

([4568, 97, 303, 4962, 589, 3783, 2642, 554, 3162, 1042  …  1081, 4040, 5522, 198, 5901, 3075, 7, 3606, 4236, 3265], [91, 353, 364, 685, 3356, 3896, 4261, 4784, 4976, 5077  …  5628, 7363, 4170, 8751, 4673, 5935, 798, 7435, 4491, 8359])

In [7]:
H[rowperm, colperm]

6595×9000 SparseMatrixCSC{Int64, Int64} with 19783 stored entries:
⠙⢮⣖⣂⡇⡐⠂⣆⢥⣰⣤⣚⡴⠗⢭⢰⣝⠰⣷⠢⡩⣓⣂⢎⢓⡧⣵⢷⡆⣿⢍⡦⣘⣀⣧⢄⡲⢦⡲⠐
⠀⠀⠙⢿⡿⣝⢲⢮⢮⣧⢛⣮⡧⡷⣽⡀⡡⠮⡒⠮⣜⣓⠛⡹⡿⠕⢯⢭⠳⣿⡐⣡⡾⠟⠯⢓⢈⡢⠽⢆
⠀⠀⠀⠀⠙⢮⡙⢿⣻⠕⢙⣼⠪⣨⡞⡙⣴⡳⣹⡻⡢⡾⢚⢁⡵⡿⢸⡗⢫⣧⠶⡅⠏⣇⡦⣌⠳⠹⠐⣨
⠀⠀⠀⠀⠀⠀⠙⢦⡈⠛⢯⣧⠫⠹⢳⡻⢏⣓⠴⣋⣼⣗⣤⣶⡘⢧⣩⡭⠾⣿⢊⠹⣽⡻⠆⢣⠹⠦⡶⣔
⠀⠀⠀⠀⠀⠀⠀⠀⠙⢦⡀⠈⠻⢷⡗⣟⣆⡓⣧⢵⡓⣎⠋⡬⣭⡰⡥⡥⣼⣹⣷⠐⢘⣫⣟⡊⣰⣃⠿⡁
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠙⢦⡀⠀⠙⠿⣮⠽⡿⠼⢌⢈⣇⡓⢹⡻⣷⠹⡻⠺⣿⡵⣫⣗⣣⡱⡳⢧⢑⡇
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠙⢦⡀⠀⠈⠳⣕⣹⣵⣍⠹⣨⢷⢱⡀⣣⠧⠀⣿⣿⣯⠵⣡⣾⡵⢔⠷⡶
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠙⢦⡀⠀⠀⠙⢮⣾⣻⠙⡾⣶⢧⡮⣨⡂⠸⣿⣿⣹⣋⣯⣻⣍⣨⠽
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠙⢦⡀⠀⠀⠙⢷⡜⣩⠆⡹⡼⠆⠀⠀⢿⣿⣧⡳⣵⡬⢍⠍⡛
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠙⢦⡀⠀⠀⠙⢶⡯⠿⣺⣐⡁⠀⠈⣿⣿⣿⣻⠗⠶⣽⠭
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠙⢦⡀⠀⠀⠹⣧⡻⠷⠂⠀⠀⠘⣿⣿⣿⣧⣏⡦⢟
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠙⢦⡀⠀⠈⢷⢗⡀⠀⠀⠀⠘⢿⣿⣿⣮⢷⡟
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠙⢦⡀⠈⢧⠀⠀⠀⠀⠀⠈⠻⣿⣿⣿⠋
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠙⢦⡘⡇⠀⠀⠀⠀⠀⠀⠘⠿⣿⣿
⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠀⠙⠃⠀⠀⠀⠀⠀⠀⠀⠀⠈⠛

In [8]:
include("../code/sp.jl")

iteration! (generic function with 1 method)

In [22]:
using StaticArrays, OffsetArrays
x0 = rand((0,1), size(H,2))
s = (-1).^x0
J = 10
sp = survey_propagation(H; field=s, init=OffsetArray(MVector{2J+1}(fill(1/(2J+1),2J+1)), -J:J), y=1.0);

In [23]:
callback(t, ε, sp) = (println("$t $ε"); false)

iteration!(sp, maxiter=10, callback=callback, tol=1e-5, damp=0.5)

1 0.9523809523809523
2 0.9523809523809523
3 0.9523809523809523
4 0.9523809523809523
5 0.9523809523809523
6 0.9523809523809523
7 0.9523809523809523
8 0.9523809523809523
9 0.9523809523809523
10 0.8317800195994005
1 0.5754866408458419
2 0.7636337558546769
3 0.5845150956830749
4 0.7645940281437071
5 0.7645940281437071
6 0.7645940281437071
7 0.6466006351731476
8 0.6015529028909068
9 0.6025558423249117
10 0.6711538216193357
1 0.5900907626427497
2 0.5900907626427497
3 0.5900907626427497
4 0.5806444249314101
5 0.5806444249314101
6 0.5918162668646194
7 0.5692361916381619
8 0.5692361916381619
9 0.5745504976997904
10 0.5745504976997904
1 0.4912131607298395
2 0.5646409602648184
3 0.5085856512014326
4 0.5085856512014326
5 0.5085856512014326
6 0.5085856512014326
7 0.4912131607298395
8 0.6038659454201463
9 0.6038659454201463
10 0.6038659454201463
1 0.4337037930044091
2 0.4383870189552237
3 0.5012138422459989
4 0.42869517010006963
5 0.4346235408988528
6 0.658312506288823
7 0.658312506288823
8 0.658312

In [12]:
x = map(x->(x'eachindex(x)<0), sp.survey);

In [13]:
sum(H*x0  .% 2)

3290

In [14]:
sum(H*x .% 2)

700

In [16]:
sum(x .!= x0)/length(x)

0.18522222222222223

In [17]:
sp.survey[1]

21-element OffsetArray(::Vector{Float64}, -10:10) with eltype Float64 with indices -10:10:
 2.3887817885620204e-63
 4.5231432433321733e-63
 1.2295097783308302e-62
 3.3438858001909284e-62
 1.3681353183204815e-44
 3.462365903682704e-28
 6.7636263317837036e-18
 3.3024481663380286e-8
 0.00013367195697020789
 0.13523157399576677
 0.8555791075316439
 0.009050125649684795
 5.487237113221181e-6
 6.043393756970963e-10
 2.3114430145466046e-23
 5.7347784226631115e-39
 9.483357447824906e-60
 5.61644386959382e-61
 7.627485325385263e-62
 9.017592036378608e-63
 6.100478815950348e-64

In [36]:
using ProgressMeter
callback(t,fg) = nothing;#println("$t $(fg.belief[1:4])")

nsamples = 10
fg = maxsum(H,  1 .- 2y .+ 1e-8*randn(N));

# the mapping is (0,1)=>(1,-1)
dist = map(1:nsamples) do i
    y = rand((0,1), N)
    fg.M .= 0
    fg.efield .= 1 .- 2y .+ 1e-8*randn(N)
    iteration!(fg; maxiter=2000, γ=1e-3, callback=callback)
    x = Int.(fg.belief .< 0)
    c = sum(fg.H'x .% 2)
    d = sum(x .⊻ y)/N
    @show d c
    x1 = B*x[indep] .% 2; d1 = sum(x1 .⊻ y)/N; c1 = sum(fg.H'x1 .% 2); @show d1 c1 
    d1
end
sum(dist)/nsamples

LoadError: UndefVarError: y not defined

In [37]:
1-M/N,sum(dist)/nsamples

LoadError: UndefVarError: dist not defined

In [283]:
x=[1:10;]; f=(x .% 2 .== 1);x[findall(f)]==x[f]

true

In [52]:
dist = map(1:10) do i
    Random.seed!(i)
    
    y = rand((0,1), N)

    lm = LossyModelGF2(fg, zeros(N), Inf, 1.0, y)
    
    function checker()
        v = (B*((fg.fields .< 0)[indep])) .% 2;
        dist = sum(v .⊻ y)/N
        check = sum(H*(fg.fields .< 0) .% 2)
        check, dist
    end

    independent = falses(N)
    independent[indep] .= true
    res_fg = solve!(lm, fg, randseed=1, verbose=true, showprogress=true, 
        independent=independent, basis = B)
#    res_fg = solve!(lm, fg, randseed=1, verbose=true, showprogress=true)
    avg_dist = res_fg.distortion 
    check, dist = checker()
    @show dist check avg_dist
    dist
end

Trial 1/1 100%|█████████████████████████████████████████| Time: 0:00:01


dist = 0.24287749287749288
check = 1
avg_dist = 0.4309116809116809


Trial 1/1 100%|█████████████████████████████████████████| Time: 0:00:01


dist = 0.24786324786324787
check = 1
avg_dist = 0.4309116809116809


Trial 1/1 100%|█████████████████████████████████████████| Time: 0:00:01


...

Trial 1/1 100%|█████████████████████████████████████████| Time: 0:00:01


dist = 0.245014245014245
check = 6
avg_dist = 0.4458689458689459
dist = 0.24715099715099714
check = 0
avg_dist = 0.24715099715099714


Trial 1/1 100%|█████████████████████████████████████████| Time: 0:00:01


dist = 0.25427350427350426
check = 2
avg_dist = 0.44515669515669515
dist = 0.24857549857549857
check = 0
avg_dist = 0.24857549857549857


10-element Vector{Float64}:
 0.24287749287749288
 0.24786324786324787
 0.25427350427350426
 0.26495726495726496
 0.2378917378917379
 0.26851851851851855
 0.245014245014245
 0.24715099715099714
 0.25427350427350426
 0.24857549857549857

In [53]:
dev = sqrt(sum(dist.^2)/length(dist)-(sum(dist)/length(dist))^2)/sqrt(length(dist))
1-M/N, sum(dist)/length(dist)

(0.24002849002849003, 0.2511396011396011)